In [ ]:
# Change the current working directory to the repo root.
# It makes the relative imports easier
import os
from pathlib import Path
from git import Repo
repo = Repo(Path().resolve(), search_parent_directories=True)
project_root = Path(repo.git.rev_parse("--show-toplevel"))
os.chdir(project_root)

# Gomoku Parameters Selection
Notebook to try different simulation parameters to find the most interesting ones.

In [11]:
import subprocess
import collections
import pandas as pd
from tqdm import tqdm
import shutil

from simulations.gomoku import GameEvent

In [ ]:
game_dir = "game_logs"
os.makedirs(game_dir, exist_ok=True) 

In [ ]:
param_sets = [
    {'W': 6, 'n': 4, 'time_limit': 0.1, 'init_blocks': 2, 'add_blocks': 2, 'b': 7},
    {'W': 7, 'n': 4, 'time_limit': 0.1, 'init_blocks': 2, 'add_blocks': 2, 'b': 7},
    {'W': 8, 'n': 4, 'time_limit': 0.1, 'init_blocks': 2, 'add_blocks': 2, 'b': 7},
    {'W': 7, 'n': 4, 'time_limit': 0.1, 'init_blocks': 2, 'add_blocks': 2, 'b': 10},
    {'W': 7, 'n': 4, 'time_limit': 0.1, 'init_blocks': 2, 'add_blocks': 2, 'b': 12},
    {'W': 7, 'n': 4, 'time_limit': 0.1, 'init_blocks': 5, 'add_blocks': 2, 'b': 10},
    {'W': 7, 'n': 4, 'time_limit': 0.01, 'init_blocks': 2, 'add_blocks': 2, 'b': 10},

]

results = {}

for params in param_sets:
    # Create a hashable key for display
    key = ", ".join(f"{k}={v}" for k, v in params.items())
    counts = collections.Counter()
    
    for i in tqdm(range(25)):
        # build the command
        cmd = ['python', 'simulations/gomoku.py']
        for k, v in params.items():
            flag = f"-{k}" if len(k) == 1 else f"--{k}"
            if isinstance(v, bool):
                if v:
                    cmd.append(flag)
            else:
                cmd += [flag, str(v)]
        # specify a unique output file so we can read it in
        out_file = os.path.join(game_dir, f"game_{key.replace(', ','_')}_{i}.txt")
        cmd += ['--output', out_file]
        # run the game
        subprocess.run(cmd, check=True)
        
        # read & parse
        with open(out_file, 'r') as f:
            text = f.read()
        data = eval(text)
        outcome = data[0][-1]   # last element of the first run’s history
        counts[outcome] += 1
    print(cmd)
    
    results[key] = counts
shutil.rmtree(game_dir)

In [5]:
df = pd.DataFrame.from_dict(results, orient='index').fillna(0).astype(int)
df = df.rename_axis('params').rename(columns={'draw': 'draw', '1': 'player1', '2': 'player2'})
print(df)

                                                    draw  player1  player2
params                                                                    
W=6, n=4, time_limit=0.1, init_blocks=2, add_bl...    17        4        4
W=7, n=4, time_limit=0.1, init_blocks=2, add_bl...     3       12       10
W=7, n=4, time_limit=0.1, init_blocks=2, add_bl...     2       13       10
W=7, n=4, time_limit=0.1, init_blocks=2, add_bl...     2       16        7
W=7, n=4, time_limit=0.1, init_blocks=5, add_bl...     8        8        9
W=7, n=4, time_limit=0.01, init_blocks=2, add_b...     3       18        4
W=8, n=4, time_limit=0.1, init_blocks=2, add_bl...     0       19        6


In [6]:
df

,draw,player1,player2
params,,,
"W=6, n=4, time_limit=0.1, init_blocks=2, add_blocks=2, b=7",17,4,4
"W=7, n=4, time_limit=0.1, init_blocks=2, add_blocks=2, b=7",3,12,10
"W=7, n=4, time_limit=0.1, init_blocks=2, add_blocks=2, b=10",2,13,10
"W=7, n=4, time_limit=0.1, init_blocks=2, add_blocks=2, b=12",2,16,7
"W=7, n=4, time_limit=0.1, init_blocks=5, add_blocks=2, b=10",8,8,9
"W=7, n=4, time_limit=0.01, init_blocks=2, add_blocks=2, b=10",3,18,4
"W=8, n=4, time_limit=0.1, init_blocks=2, add_blocks=2, b=7",0,19,6


It seems that `W=7, n=4, time_limit=0.1, init_blocks=2, add_blocks=2, b=7` is a good combination that rarely leads to draw, but might prefer player1 over player2. Another interesting one is `W=7, n=4, time_limit=0.1, init_blocks=5, add_blocks=2, b=10` because it has the same number of draws as players wins. I'll collect more statistics on them.

In [ ]:
game_dir = "game_logs"
os.makedirs(game_dir, exist_ok=True) 

param_sets = [
    {'W': 7, 'n': 4, 'time_limit': 0.1, 'init_blocks': 2, 'add_blocks': 2, 'b': 7},
    {'W': 7, 'n': 4, 'time_limit': 0.1, 'init_blocks': 5, 'add_blocks': 2, 'b': 10},

]

results = {}

for params in param_sets:
    # Create a hashable key for display
    key = ", ".join(f"{k}={v}" for k, v in params.items())
    counts = collections.Counter()
    
    for i in tqdm(range(50)):
        # build the command
        cmd = ['python', 'simulations/gomoku.py']
        for k, v in params.items():
            flag = f"-{k}" if len(k) == 1 else f"--{k}"
            if isinstance(v, bool):
                if v:
                    cmd.append(flag)
            else:
                cmd += [flag, str(v)]
        # specify a unique output file so we can read it in
        out_file = os.path.join(game_dir, f"game_{key.replace(', ','_')}_{i}.txt")
        cmd += ['--output', out_file]
        # run the game
        subprocess.run(cmd, check=True)
        
        # read & parse
        with open(out_file, 'r') as f:
            text = f.read()
        data = eval(text)
        outcome = data[0][-1]   # last element of the first run’s history
        counts[outcome] += 1
    print(cmd)
    
    results[key] = counts
shutil.rmtree(game_dir)

In [8]:
df = pd.DataFrame.from_dict(results, orient='index').fillna(0).astype(int)
df = df.rename_axis('params').rename(columns={'draw': 'draw', '1': 'player1', '2': 'player2'})
df

,player1,player2,draw
params,,,
"W=7, n=4, time_limit=0.1, init_blocks=2, add_blocks=2, b=7",24,19,7
"W=7, n=4, time_limit=0.1, init_blocks=5, add_blocks=2, b=10",15,13,22


It seems that `init_blocks=5, b=10` leads to more balanced games, but it has significantly more draws. Let's try the intermediate options. 

In [ ]:
game_dir = "game_logs"
os.makedirs(game_dir, exist_ok=True) 

param_sets = [
    {'W': 7, 'n': 4, 'time_limit': 0.1, 'init_blocks': 3, 'add_blocks': 2, 'b': 10},
    {'W': 7, 'n': 4, 'time_limit': 0.1, 'init_blocks': 4, 'add_blocks': 2, 'b': 10},

]

results = {}

for params in param_sets:
    # Create a hashable key for display
    key = ", ".join(f"{k}={v}" for k, v in params.items())
    counts = collections.Counter()
    
    for i in tqdm(range(50)):
        # build the command
        cmd = ['python', 'simulations/gomoku.py']
        for k, v in params.items():
            flag = f"-{k}" if len(k) == 1 else f"--{k}"
            if isinstance(v, bool):
                if v:
                    cmd.append(flag)
            else:
                cmd += [flag, str(v)]
        # specify a unique output file so we can read it in
        out_file = os.path.join(game_dir, f"game_{key.replace(', ','_')}_{i}.txt")
        cmd += ['--output', out_file]
        # run the game
        subprocess.run(cmd, check=True)
        
        # read & parse
        with open(out_file, 'r') as f:
            text = f.read()
        data = eval(text)
        outcome = data[0][-1]   # last element of the first run’s history
        counts[outcome] += 1
    print(cmd)
    
    results[key] = counts
shutil.rmtree(game_dir)

In [10]:
df = pd.DataFrame.from_dict(results, orient='index').fillna(0).astype(int)
df = df.rename_axis('params').rename(columns={'draw': 'draw', '1': 'player1', '2': 'player2'})
df

,player1,player2,draw
params,,,
"W=7, n=4, time_limit=0.1, init_blocks=3, add_blocks=2, b=10",28,16,6
"W=7, n=4, time_limit=0.1, init_blocks=4, add_blocks=2, b=10",19,18,13


Alright, I like `W=7, n=4, time_limit=0.1, init_blocks=4, add_blocks=2, b=10`. It's outcome balanced.